# Import

In [ ]:
import json
import cv2
import pandas as pd
import numpy as np

In [ ]:
id_joints_dict = {0: 'nose',
        1: 'left_eye',
        2: 'right_eye',
        3: 'left_ear',
        4: 'right_ear',
        5: 'left_shoulder',
        6: 'right_shoulder',
        7: 'left_elbow',
        8: 'right_elbow',
        9: 'left_wrist',
        10: 'right_wrist',
        11: 'left_hip',
        12: 'right_hip',
        13: 'left_knee',
        14: 'right_knee',
        15: 'left_ankle',
        16: 'right_ankle'}
joints_id_dict = {v: k for k, v in id_joints_dict.items()}

In [ ]:
import re

def format_action(action_filename):
    '''
    Transform the name of an action file into the format "Subject.X/Action"

    Args:
        action_filename (str): the name of the action file
        
    Returns:
        str: the path to the action file in the format "Subject.X/Action"
    '''


    # Récupérer le numéro de sujet
    subject_number = re.search(r'S(\d+).json$', action_filename).group(1) # group(1) pour récupérer le premier groupe de la regex, qui correspond au numéro de sujet car c'est ce qui est entre parenthèses
    # Supprimer l'extension .json et le "S" suivi du numéro à la fin
    action = re.sub(r'S\d+.json$', '', action_filename)
    # Cas d'exception pour "Pickupobject"
    if action == "Pickupobject":
        formatted_action = "Pick up object"
    else:
        # Séparer les mots en utilisant les majuscules comme séparateur
        words = re.findall('[A-Z][^A-Z]*', action)
        # Mettre la première en minuscule, sauf pour le premier mot
        words = [words[0]] + [word[0].lower() + word[1:] for word in words[1:]]
        # Joindre les mots avec un espace
        formatted_action = ' '.join(words)
    return f'Subject.{subject_number}/{formatted_action}'

# Testons le script avec vos exemples
print(format_action('FallBackwardsS1.json'))  # 'Subject.1/Fall backwards'
print(format_action('HopS2.json'))  # 'Subject.2/Hop'
print(format_action('PickupobjectS3.json'))  # 'Subject.3/Pick up object'

In [ ]:
def get_video_info_dict(video_path):
    cap = cv2.VideoCapture(video_path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    codec = "".join([chr((fourcc >> 8 * i) & 0xFF) for i in range(4)])
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) / int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()
    return {'num_frames': num_frames, 'fps': fps, 'codec': codec, 'width': width, 'height': height, 'duration': duration}

In [ ]:
import os

list_dir = os.listdir('../../outputs/fall/bbox_imposed/predictions/')
list_dir.sort()

for action_filename in list_dir:
    print(format_action(action_filename))


## Load pose sequence

In [ ]:
file_path = '../../outputs/fall/bbox_imposed/predictions/FallBackwardsS3.json'
with open(file_path) as json_file:
    pose_sequence = json.load(json_file)

In [ ]:
print(len(pose_sequence))
pose_sequence[0].keys()

## Process the data

### Convert into a Series

we convert the json in a dataframe. Each frame is a row in the dataframe. 

In [ ]:
pose_sequence[0]['instances'][0]['keypoints']

In [ ]:
keypoints_dict = {}
keypoints_list = pose_sequence[0]['instances'][0]['keypoints']

for number, keypoint in enumerate(keypoints_list):
    keypoints_dict["X_" + id_joints_dict[number]] = keypoint[0]
    keypoints_dict["Y_" + id_joints_dict[number]] = keypoint[1]
print(keypoints_dict)


In [ ]:
ser = pd.Series(data=keypoints_dict)
ser

### Convert into a dataframe

In [ ]:
dic_list = []

for i in range(2):
    keypoints_dict = {}
    keypoints_list = pose_sequence[i]['instances'][0]['keypoints']

    for number, keypoint in enumerate(keypoints_list):
        keypoints_dict["X_" + id_joints_dict[number]] = keypoint[0]
        keypoints_dict["Y_" + id_joints_dict[number]] = keypoint[1]
    
    dic_list.append(keypoints_dict)

In [ ]:
pd.DataFrame.from_dict(dic_list)

It worked, so now we gonna do it for the whole file

In [ ]:
pose_sequence[90]

In [ ]:
dic_list = []

for i in range(len(pose_sequence)):
    print(i)
    keypoints_dict = {}
    keypoints_list = pose_sequence[i]['instances'][0]['keypoints']

    for number, keypoint in enumerate(keypoints_list):
        keypoints_dict["X_" + id_joints_dict[number]] = keypoint[0]
        keypoints_dict["Y_" + id_joints_dict[number]] = keypoint[1]
    
    dic_list.append(keypoints_dict)
    
df = pd.DataFrame.from_dict(dic_list)

In [ ]:
print(len(df))
df

### Now we add the labell 

In [ ]:
from prodigy.components.db import connect

db = connect()
fall_data = db.get_dataset("fall_data")

def get_fall_label(file_path):
    for i in range(len(fall_data)):
        if fall_data[i]['text'] == file_path.split('/')[-1].split('.')[0]:
            return fall_data[i]

In [ ]:
dataset_path = '../../Data/Fall/Dataset_CAUCAFall/CAUCAFall/'
images_path = format_action(file_path.split("/")[-1])
images_list = os.listdir(dataset_path + images_path)
labels_file_list = [file for file in images_list if file.endswith(".txt")]
labels_file_list.sort()
labels_file_list.remove("classes.txt")

In [ ]:
file_path

In [ ]:
labels_list = []

for file in labels_file_list:
    with open(dataset_path + images_path + '/' + file) as f:
        class_integer = int(f.readline()[0])
        if class_integer == 0:
            labels_list.append('Normal')
        elif class_integer == 1:
            labels_list.append('Lying down')

labels_ser = pd.Series(labels_list, name='label')

# On ajoute les chuttes s'il y en a
label_dict = get_fall_label(file_path)

if label_dict['audio_spans']:
    # On récupère les informations de la vidéo, notamment le nombre d'images par seconde
    video_folder_path = '../../Data/Fall/Dataset_CAUCAFall/video/'
    info_dict = get_video_info_dict(video_folder_path + label_dict['text'] + '.mp4')
    
    start_frame = round(label_dict['audio_spans'][0]['start'] * info_dict['fps'])
    end_frame = round(label_dict['audio_spans'][0]['end'] * info_dict['fps'])
    labels_ser[start_frame:end_frame] = 'Fall'
labels_ser

### We add up everything in a final function

In [ ]:
def get_df_from_preds(file_path):

    with open(file_path) as json_file:
        pose_sequence = json.load(json_file)
    
    dic_list = []

    for i in range(len(pose_sequence)):
        keypoints_dict = {}

        try:
            keypoints_list = pose_sequence[i]['instances'][0]['keypoints']
        except:
            keypoints_list = [[np.nan, np.nan] for i in range(17)]

        for number, keypoint in enumerate(keypoints_list):
            keypoints_dict["X_" + id_joints_dict[number]] = keypoint[0]
            keypoints_dict["Y_" + id_joints_dict[number]] = keypoint[1]
        
        dic_list.append(keypoints_dict)
        
    df = pd.DataFrame.from_dict(dic_list)

    dataset_path = '../../Data/Fall/Dataset_CAUCAFall/CAUCAFall/'
    images_path = format_action(file_path.split("/")[-1])
    images_list = os.listdir(dataset_path + images_path)
    labels_file_list = [file for file in images_list if file.endswith(".txt")]
    labels_file_list.sort()
    labels_file_list.remove("classes.txt")

    labels_list = []

    for file in labels_file_list:
        with open(dataset_path + images_path + '/' + file) as f:
            class_integer = int(f.readline()[0])
            if class_integer == 0:
                labels_list.append('Normal')
            elif class_integer == 1:
                labels_list.append('Lying down')

    labels_ser = pd.Series(labels_list, name='label')
    # On ajoute les chuttes s'il y en a
    label_dict = get_fall_label(file_path)

    if label_dict['audio_spans']:
        # On récupère les informations de la vidéo, notamment le nombre d'images par seconde
        video_folder_path = '../../Data/Fall/Dataset_CAUCAFall/video/'
        info_dict = get_video_info_dict(video_folder_path + label_dict['text'] + '.mp4')
        
        start_frame = round(label_dict['audio_spans'][0]['start'] * info_dict['fps'])
        end_frame = round(label_dict['audio_spans'][0]['end'] * info_dict['fps'])
        labels_ser[start_frame:end_frame] = 'Fall'

    df = pd.concat([df, labels_ser], axis=1)


    return df

In [ ]:
get_df_from_preds(file_path=file_path)

Now, we apply this function to all predictions and save them as  csv files.

In [ ]:
preds_folder_path = '../../outputs/fall/bbox_imposed/predictions/'
preds_files = os.listdir(preds_folder_path)
preds_files.sort()

In [ ]:
output_folder_path = '../../Data/Fall/Dataset_CAUCAFall/Pose_sequences/'

for preds_file in preds_files:
    df = get_df_from_preds(preds_folder_path + preds_file)
    df.to_csv(output_folder_path + preds_file.replace(".json", ".csv"))
